In [1]:
import numpy as np
from model_functions import *
from Classification_models_LOSO_2Class import *

In [ ]:
#uncomment to run and create the all_psds file. 

#subject_data, freqs, targets = load_data(duration=30,overlap=15,seg_length=4)
#save_psds(subject_data,freqs,targets,'all_psds.pickle')


In [2]:
subject_data, freqs, targets = load_psds('all_psds.pickle')
test_labels = align_test_labels(classes=['A','C'])
features, targets = remove_class(subject_data,targets,'F')
features, targets = remove_test(features,targets,test_labels)
rbps = [relative_band_power(features[i],freqs,freq_bands=[0.5,4.0,7.0,9.0,11.0,13.0,25.0,45.0]) 
        for i in range(len(features))]

### k nearest neighbor classifier

In [5]:
train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = 0)
train_metric_dict, val_metric_dict

({'acc': 0.977195196037198,
  'sens': 0.9817202669902912,
  'spec': 0.9719025651424515,
  'f1': 0.9789093816515944},
 {'acc': 0.671573438011122,
  'sens': 0.6644417475728155,
  'spec': 0.6799148332150461,
  'f1': 0.6856606136505948})

In [6]:
train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = 133)
train_metric_dict, val_metric_dict

({'acc': 0.977195196037198,
  'sens': 0.9817202669902912,
  'spec': 0.9719025651424515,
  'f1': 0.9789093816515944},
 {'acc': 0.671573438011122,
  'sens': 0.6644417475728155,
  'spec': 0.6799148332150461,
  'f1': 0.6856606136505948})

In [ ]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

### Random Forest classifier

Here is a run of random Forest without doing PCA. As you can see in the definition of the method, PCA = 0, means we do not do PCA

In [3]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = 0)
train_metric_dict, val_metric_dict

({'acc': 0.9907589139679425,
  'sens': 0.995308165742025,
  'spec': 0.9854380006083342,
  'f1': 0.991462124646504},
 {'acc': 0.7487733071638861,
  'sens': 0.7870145631067961,
  'spec': 0.7040454222853088,
  'f1': 0.7715645449137418})

Here is a run of random Forest with PCA. The maximum number of features is 133 here. 
As you can see, there is a significant reduction in validation set accuracy (from 75% to 71% )

In [4]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = 133)
train_metric_dict, val_metric_dict

({'acc': 0.9991237908313473,
  'sens': 0.9999024791955617,
  'spec': 0.9982130183514144,
  'f1': 0.9991879073997878},
 {'acc': 0.7101733725875041,
  'sens': 0.7548543689320388,
  'spec': 0.6579134137686302,
  'f1': 0.7374036751630114})

In [ ]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]